In [85]:
import os
import sys
sys.path.append("../")
sys.path.append("../src")

import lmdb
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

import caffe
from experiment import top_30_class as experiment
from mit_data import MITDataset

In [86]:
dataset = MITDataset(split_file=experiment.SPLIT_FILE, index_file=experiment.INDEX_FILE)
index = dataset.index
mlb = dataset.mlb
bin_labels = dataset.binary_label

In [87]:
def get_video_file(direc, filename):
    return os.path.join(direc, filename)

In [88]:
dataset.index[0]

{'category': 'breaking',
 'directory': '../data/MIT_data/training/breaking',
 'filename': 'v018_0048.mp4',
 'object_label': 'hand',
 'object_label_list': ['hand'],
 'scene_label': nan,
 'split': 'train'}

make pair of (image, binarized-label) for whole dataset.

image should in (C x W x H) format and BGR

In [17]:
for i in range(dataset)

SyntaxError: invalid syntax (<ipython-input-17-3fe9e32b8052>, line 1)

In [92]:
i = 0
video_path = get_video_file(index[i]["directory"], index[i]["filename"])
video = dataset.load_video(video_path)[:, :, :, ::-1]
video = video.transpose([0, 3, 1, 2])
labels = bin_labels[i]

In [94]:
video.nbytes

17694720

In [98]:
map_size = np.zeros((3, 256, 256), dtype=np.uint8).nbytes * 5000000


In [109]:
env = lmdb.open('../data/MIT_data/video_lmdb', map_size=map_size)
i = 0

with env.begin(write=True) as txn:
    # txn is a Transaction object
    for i in range(len(dataset)):
        video_path = get_video_file(index[i]["directory"], index[i]["filename"])
        video = dataset.load_video(video_path)[:, :, :, ::-1]
        video = video.transpose([0, 3, 1, 2])
        labels = bin_labels[i]
        for frame in range(len(video)):
            frame = video[frame]
            
            datum = caffe.proto.caffe_pb2.Datum()
            datum.channels = frame.shape[0]
            datum.height = frame.shape[1]
            datum.width = frame.shape[2]
            datum.data = frame.tobytes()  # or .tostring() if numpy < 1.9
            str_id = '{:08}'.format(i)
            i += i + 1
            txn.put(str_id, datum.SerializeToString())

In [115]:
env = lmdb.open('../data/MIT_data/label_lmdb', map_size=map_size)
idx = 0

with env.begin(write=True) as txn:
    # txn is a Transaction object
    for i in range(len(dataset)):
        video_path = get_video_file(index[i]["directory"], index[i]["filename"])
        video = dataset.load_video(video_path)[:, :, :, ::-1]
        video = video.transpose([0, 3, 1, 2])
        labels = bin_labels[i].reshape([-1,1,1])
        for frame in range(len(video)):
            dat = caffe.io.array_to_datum(labels)
            str_id = '{:08}'.format(idx)
            idx = idx + 1
            txn.put(str_id, dat.SerializeToString())

In [114]:
labels.reshape([-1,1,1]).shape

(20, 1, 1)

In [101]:
i

813326605810494840608724615167L

In [103]:
657 * 90

59130

In [ ]:
len()